In [ ]:
# split users ratings into training, test and elite test (only high ratings) datasets
# user_datasets = {}

# for user in Users:
   
#     userMoviesTraining, userMoviesTest, full_test_set, all_movies = utils.getUserTrainingTestMovies(conn, user[0])
        
#     if len(userMoviesTest) == 0:
#         continue
    
#     random_movies = utils.getRandomMovieSet(conn, user[0])        
#     user_datasets[user[0]] = {'all_movies': all_movies, 'train': userMoviesTraining, 'elite_test': userMoviesTest, 'test': full_test_set, 'random': random_movies}

# print "Test Dataset", user_datasets[user[0]]['test'], len(user_datasets[user[0]]['test']), "\n\n"
# print "Elite Test Dataset", user_datasets[user[0]]['elite_test'], len(user_datasets[user[0]]['elite_test']), "\n\n"

In [ ]:
# LOW LEVEL FEATURES preprocess users profiles - Support Vector Machine Regressor
# user_profiles_low_level, mae = train_user_profile_svm_regressor(conn, user_datasets, LOW_LEVEL_FEATURES)
# print "Low-Level Features MAE", mae

# # DEEP FEATURES preprocess users profiles - SVM
# user_profiles_deep, mae = train_user_profile_svm_regressor(conn, user_datasets, DEEP_FEATURES)
# print "Deep Features MAE", mae

# user_profiles = recommender_classifier.build_user_profiles(conn, Users)

In [ ]:
def train_user_profile_svm_regressor(conn, user_datasets, feature_vector):
    
    user_profiles = {}
    sum_mae = 0

    for user, datasets in user_datasets.iteritems():     
        
        userInstances, userValues = utils.getUserInstances(datasets['train'], feature_vector)
        userBaseline = utils.getUserBaseline(conn, user)
        
        clf = svm.SVR(kernel='rbf')
        clf.fit(userInstances, userValues)
        
        user_profiles[user] = {'model': clf.predict, 'datasets': datasets, 'baseline': userBaseline}
        # userInstances, userValues = utils.getUserInstances(full_test_set, LOW_LEVEL_FEATURES)

        # check the model quality using user's full test set (ratings not used for the model)
        # predictions = [(movie[2], clf.predict([feature_vector[movie[0]]])) for movie in full_test_set]
        # print predictions
        
        real_ratings = [x[1] for x in datasets['test']]
        predicted_ratings = [clf.predict([feature_vector[movie[0]]]) for movie in datasets['test']]
        
        # sum_mae += evaluation.evaluateMAE(conn, user, predictions, 0, 1)
        sum_mae += mean_absolute_error(real_ratings, predicted_ratings)

    mae = utils.evaluateAverage(sum_mae, len(user_datasets))
    return user_profiles, mae

In [1]:
# necessary modules
import sqlite3
import utils
import recommender_classifier
import evaluation
import time
from sklearn import svm
from sklearn.metrics import mean_absolute_error
from multiprocessing import Process, Queue

In [2]:
# load random users and feature vectors
conn = sqlite3.connect('database.db')
Users = utils.selectRandomUsers(conn)
LOW_LEVEL_FEATURES, DEEP_FEATURES, HYBRID_FEATURES = utils.extract_features()

print len(Users)

user_profiles = recommender_classifier.build_user_profiles(conn, Users)

60


In [3]:
def run(user_profiles, N, feature_vector):

    conn = sqlite3.connect('database.db')
    
    SumRecall, SumPrecision = 0, 0
    
    for user, profile in user_profiles.iteritems():                
        
        hits = 0
        
        predictions = recommender_classifier.get_predict_collaborative_filtering(conn, profile, feature_vector)
        # print "Predictions", sorted(predictions, key=lambda tup: tup[2], reverse=True)
        
        for elite_movie in profile['datasets']['elite_test']:
                        
            if feature_vector is list and elite_movie[0] not in feature_vector:
                continue
                
            # Predict to the user movie and to random movies that the user did not rated
            # print predictions            
            elite_prediction = recommender_classifier.get_prediction_elite(conn, elite_movie, profile, feature_vector)
            all_predictions = predictions[:]
            all_predictions.append(elite_prediction)
            
            # print "Elite Movie", elite_movie, elite_prediction
            
            hits += recommender_classifier.count_hit(all_predictions, elite_movie, N)
        try:
            recall = hits / float(len(profile['datasets']['elite_test']))
            SumRecall += recall
            SumPrecision += (recall / float(N))
        except ZeroDivisionError:
            continue
        # print "Size is", len(predictions)
        # print "Predictions", sorted(predictions, key=lambda tup: tup[2], reverse=True)

    size = len(user_profiles)
    avgRecall = utils.evaluateAverage(SumRecall, size)
    avgPrecision = utils.evaluateAverage(SumPrecision, size)

    return avgPrecision, avgRecall

In [7]:
# def experiment(N, user_profiles_low_level, LOW_LEVEL_FEATURES, user_profiles_deep, DEEP_FEATURES):
def experiment(N):
    
    global user_profiles, LOW_LEVEL_FEATURES, DEEP_FEATURES, HYBRID_FEATURES
            
    result = {}

    # COLLABORATIVE FILTERING
#     p_c, r_c = run(user_profiles, N, None)
#     print "CF Recall", r_c, "CF Precision", p_c, "For iteration with", N, "\n\n"
#     result[N] = {'cf': {'recall': r_c, 'precision': p_c}}
    
    # LOW LEVEL FEATURES check precision, recall and mae
    p_l, r_l = run(user_profiles, N, LOW_LEVEL_FEATURES)
    print "Low-Level Recall", r_l, "Low-Level Precision", p_l, "For iteration with", N, "\n\n"
#     result[N] = {'ll': {'recall': r_l, 'precision': p_l}}

    # DEEP FEATURES check precision, recall and mae
    p_d, r_d = run(user_profiles, N, DEEP_FEATURES)
    print "Deep Recall", r_d, "Deep Precision", p_d, "For iteration with", N, "\n\n"
#     result[N] = {'deep': {'recall': r_d, 'precision': p_d}}
    
    # HYBRID
#     p_d, r_d = run(user_profiles_deep, N, HYBRID_FEATURES)
#     p_d, r_d = run(user_profiles, N, HYBRID_FEATURES)
#     print "Hybrid Recall", r_d, "Hybrid Precision", p_d, "For iteration with", N, "\n\n"
#     result[N] = {'hybrid': {'recall': r_d, 'precision': p_d}}

    p, r, mae = recommender_classifier.recommend_random(user_profiles, N)
    print "Random Recall", r, "Random Precision",  p, "Random MAE", mae, "For iteration with", N
    
#     result = {'ll': {'recall': r_l, 'precision': p_l}, 'deep': {'recall': r_d, 'precision': p_d}, 'random': {'recall': r, 'precision': p}}
   

In [8]:
q = Queue()
N = 10
start = time.time()
# experiment(N, user_profiles, LOW_LEVEL_FEATURES, DEEP_FEATURES, HYBRID_FEATURES)
# procs = [Process(target=experiment, args=(N, user_profiles, LOW_LEVEL_FEATURES, DEEP_FEATURES, HYBRID_FEATURES,)) for i in range(1)]
# for p in procs: p.start()
# for p in procs: p.join()
end = time.time()
print(end - start)

#     p = Process(target=experiment, args=(q, N, user_profiles_low_level, LOW_LEVEL_FEATURES, user_profiles_deep, DEEP_FEATURES, Users,))
#     p.start()
#     N += 1
# print q.get()
# p.join()

p = Pool(5)
print(p.map(run, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))

Low-Level Recall 0.428271964732 Low-Level Precision 0.0428271964732 For iteration with 10 


Deep Recall 0.474708711201 Deep Precision 0.0474708711201 For iteration with 10 


Random Recall 0.0 Random Precision 0.0 Random MAE 1.31053255668 For iteration with 10
User Avg Recall 0.0 User Avg Precision 0.0 User Avg MAE 1.31053255668 For iteration with 10
541.164047003
